# 이미지 클러스터

1. 이미지를 RGB순으로 읽어오기
2. 군집수를 지정하여 Kmeans라는 평균 알고리즘으로 비슷한 색끼리 모은 군집을 지정된 수만큼 만들어냄
3. 픽셀의 숫자를 기반으로 히스토그램 형식으로 색을 반환
4. 각 색의 빈도를 나타내는 바를 반환

In [ ]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import scipy.misc

In [ ]:
def centroid_histogram(clt):
    '''
    # grab the number of different clusters and create a histogram
    히스토그램 형식으로 색을 반환
    based on the number of pixels assigned to each cluster
    각 클러스터의 픽셀의 숫자를 기반으로 함
    '''
    numLabels = np.arange(0, len(np.unique(clt.labels_)) + 1)
    (hist, _) = np.histogram(clt.labels_, bins=numLabels)

    # normalize the histogram, such that it sums to one
    hist = hist.astype("float")
    hist /= hist.sum()  # hist = hist/hist.sum()

    # return the histogram
    return hist

In [ ]:
def plot_colors(hist, centroids):
    '''
    initialize the bar chart representing the relative frequency of each of the colors
    각 색의 빈도를 나타내는 바 차트를 초기화
    '''
    bar = np.zeros((50, 300, 3), dtype="uint8")
    startX = 0

    # loop over the percentage of each cluster and the color of each cluster
    for (percent, color) in zip(hist, centroids):
        # plot the relative percentage of each cluster
        endX = startX + (percent * 300)
        cv2.rectangle(bar, (int(startX), 0), (int(endX), 50),
                      color.astype("uint8").tolist(), -1)
        startX = endX

    # return the bar chart
    return bar

In [ ]:
def image_color_cluster(image_path, k = 5):
    image = cv2.imread(image_path)
    # image의 shape을 찍어보면, height, width, channel 순으로 나옴
    # channel은 RGB를 말함
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # cv에서는 RGB가 아닌 BGR 순으로 나오기 때문에 순서를 RGB로 전환
    image = image.reshape((image.shape[0] * image.shape[1], 3))
    # shape의 0,1번째 즉, height와 width를 통합시킴

    clt = KMeans(n_clusters = k)  # 평균 알고리즘 KMeans
    clt.fit(image)

    hist = centroid_histogram(clt)
    bar = plot_colors(hist, clt.cluster_centers_)
    return bar

# DataFrame 만들기

1. 매장 외부와 음식사진을 제외한 내부 인테리어가 잘 나온 이미지 사진만을 가져오기 위해서 크롤러 대신 수작업을 선택했습니다.
2. 각자의 구역을 대분류(도심, 서서울, 남서울 등)와 소분류(은평구, 마포구, 동작구 등)로 나누어 하루 10개씩 작업하였습니다.
3. 모든 이미지 파일은 **'대분류\_소분류\_인덱스.png'**\로 통일했습니다.

![구글스프레드시트01](google_spreadsheet_01.png)

4. 이미지 수집을 끝내야하는 시점에는 가장 많은 수의 이미지를 가져온 사람을 기준으로 수를 통일했습니다.(지역별 13개씩)

![구글스프레드시트02](google_spreadsheet_02.png)

5. 스프레드시트에서 제공하는 기능을 이용해 위도와 경도를 구했습니다.
6. 이를 한데 모아 csv로 파일로 저장했습니다.

저장한 이미지들을 모은 폴더입니다.

![이미지폴더](cafe_image_folder.png)

In [ ]:
df_cafe = pd.read_csv('final_cafe_info.csv')
df_cafe = df_cafe.drop('Unnamed: 6', axis=1)

df_cafe.head(10)

이미지 파일명을 각각에 지정했습니다.

In [ ]:
area_names = {'ws' : ['ep', 'mp', 'sdm'],
              'sw' : ['gs', 'ys', 'ydp', 'gr'],
              'cs' : ['jr', 'jg', 'ys'],
              'ss' : ['dj', 'ga', 'gc'],
              'gn' : ['sc', 'gn'],
              'gb' : ['db', 'gb', 'sb', 'nw'],
              'es' : ['ddm', 'jl', 'sd', 'gj'],
              'se' : ['gd', 'sp']
             }

path_list = ['{0}_{1}_0{2}.png'.format(k, v, i) # 10부터는 어떻게 나타내지.. 고민해보자 소령
            for k in area_names.keys()
            for v in area_names[k]
            for i in range(1,14)]

path_list[:10]

In [ ]:
df_cafe['파일명'] = path_list
df_cafe.head(10)

In [ ]:
df_cafe.to_csv('final_cafe_info_with_path.csv')

# 이미지 클러스터 실행

정리된 data를 바탕으로 자동으로 이미지 클러스터를 돌릴 수 있도록 했습니다.

컴퓨터가 자꾸 열을 받아서 중간중간 쉬어가매 약 3일 정도를 돌렸습니다...(장비병.....)

In [ ]:
'''
이거슨 매우 위험한 코드입니다. 잘못하면 컴퓨터 터질지경
for index in range(len(path_list)):
    result = image_color_cluster('./cafe_image/'+path_list[index])
    scipy.misc.imsave('./cafe_color_result/'+path_list[index], result)
'''

아래 사진은 결과를 저장한 폴더입니다.

![결과이미지](cafe_color_result_folder.png)

# 대표색
(5개의 색 중에서 가장 많이 나온 색)을 찾아 16진수로 바꾸어 반환하는 함수

In [ ]:
def dec_to_hex(color):
    if color < 16:
        return '0' + str(hex(int(color)).split('x')[1])
    else:
        return str(hex(int(color)).split('x')[1])

In [ ]:
def read_real_color(filename):
    image = cv2.imread(filename, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    image_list = [str(list(image[i][k])) for i in range(len(image)) for k in range(len(image[0]))]
    image_unique = {}
    for d in image_list:
        if d not in image_unique:
            image_unique[d] = 1
        else:
            image_unique[d] += 1

    import operator
    icon_color_list = max(image_unique.items(), key=operator.itemgetter(1))[0]

    color_R = int(icon_color_list.split('[')[1].split(']')[0].split(', ')[0])
    color_G = int(icon_color_list.split('[')[1].split(']')[0].split(', ')[1])
    color_B = int(icon_color_list.split('[')[1].split(']')[0].split(', ')[2])

    color_R = dec_to_hex(color_R)
    color_G = dec_to_hex(color_G)
    color_B = dec_to_hex(color_B)

    return str(color_R + color_G + color_B)

In [ ]:
color_list = []
for n in df_cafe.index:
    png = './cafe_color_result/' + df_cafe['파일명'][n]
    color_list.append(read_real_color(png))
df_cafe['대표색'] = color_list

In [ ]:
# df_cafe.to_csv('cafe_color_tidy_data.csv')

cv2설치가 필요하기 때문에 저장한 데이터를 불러와 보여드립니다.

In [2]:
df_cafe_with_color = pd.read_csv('final_cafe_color_tidy_data.csv')
df_cafe_with_color = df_cafe_with_color.drop('Unnamed: 0', axis=1)
df_cafe_with_color.head()

,지역,지역구,카페명,주소,위도,경도,파일명,1번 색,2번 색,3번 색,4번 색,5번 색
0,서서울,은평,스모어,서울 은평구 연서로29길 8-8,37.618952,126.919697,ws_ep_01.png,dbd6cc,c7b29a,231a14,ab8f73,644f3f
1,서서울,은평,필라멘트카페,서울 은평구 통일로 883,37.621753,126.919462,ws_ep_02.png,201d1e,9d7653,5f4939,cead7d,e7e4cf
2,서서울,은평,HUGA,서울 은평구 진관2로 19,37.634753,126.919774,ws_ep_03.png,36a17c,8ebda8,e5e7df,6f6c49,272b15
3,서서울,은평,YM Coffee Project,서울 은평구 연서로29길 21-8,37.619095,126.917542,ws_ep_04.png,cab9ab,e9dfd8,403126,af8b6d,84634c
4,서서울,은평,카페달력,서울 은평구 연서로18길 28-2,37.611930,126.920534,ws_ep_05.png,e8ddd1,d3c0af,b19c89,5a432c,917457


# Folium 지도
가장 많이 나온 색을 지도의 아이콘에 적용해 찍었습니다.

In [ ]:
import base64
import folium

map = folium.Map(location=[df_cafe['위도'].mean(), df_cafe['경도'].mean()], zoom_start=13)

for n in df_cafe.index:
    png = './cafe_color_result/' + df_cafe['파일명'][n]
    encoded = base64.b64encode(open(png, 'rb').read()).decode('utf-8')
    cafe_name = df_cafe['카페명'][n] + ' - ' + df_cafe['주소'][n]
    html = f'<p>{cafe_name}</p> <img src="data:image/png;base64,{encoded}">'
    iframe = folium.IFrame(html, width=700, height=130)
    popup = folium.Popup(iframe, max_width=300)
    color = '#' + df_cafe['대표색'][n]
    icon = folium.Icon(icon_color=color, color='white')
    folium.Marker([df_cafe['위도'][n], df_cafe['경도'][n]], popup=popup, icon=icon).add_to(map)

# 혹시 모를 재가공을 위한 Tidy Data
저희가 임으로 바꾼 영어 지역명이 각각 무엇인지를 보여주기 위한 DataFrame도 만들었습니다.

한글순, 영어순으로 정렬한 것을 각각 만들었습니다.

추후 html 홈페이지를 만들때 일일이 작성하기 보다는 이 data를 활용해 반복문을 만들어 손쉽게 작업할 수 있었습니다.

In [ ]:
df_sorted_area_by_kor = pd.DataFrame()

In [ ]:
area_dict = {\
'은평구': 'ws_ep', '마포구': 'ws_mp', '서대문구': 'ws_sdm', '강서구': 'sw_gs',\
'양천구': 'sw_yc', '영등포구': 'sw_ydp', '구로구': 'sw_gr', '종로구': 'cs_jr',\
'중구': 'cs_jg', '용산구': 'cs_ys', '동작구': 'ss_dj', '관악구': 'ss_ga',\
'금천구': 'ss_gc', '서초구': 'gn_sc', '강남구': 'gn_gn', '도봉구': 'gb_db',\
'강북구': 'gb_gb', '성북구': 'gb_sb', '노원구': 'gb_nw', '동대문구': 'es_ddm',\
'중랑구': 'es_gl', '성동구': 'es_sd', '광진구': 'es_gj', '강동구': 'se_gd', '송파구': 'se_sp'}

## 한글(가나다)순으로 정렬하기

In [ ]:
sorted_area_by_kor = sorted(area_dict.items(), key=lambda x:x[0])
sorted_area_by_kor[:10]

In [ ]:
area_in_korean = [area[0] for area in sorted_area_by_kor]
area_in_korean[:10]

In [ ]:
df_sorted_area_by_kor['지역명(한글)'] = area_in_korean
df_sorted_area_by_kor.head(10)

In [ ]:
area_in_english = [area[1] for area in sorted_area_by_kor]
df_sorted_area_by_kor['지역명(영문)'] = area_in_english

In [ ]:
df_sorted_area_by_kor.head()

In [ ]:
df_sorted_area_by_kor.to_csv('data_sorted_area_by_kor.csv')

## 같은 방법으로 영문순으로 정렬하기

In [ ]:
df_sorted_area_by_eng = pd.DataFrame()

In [ ]:
sorted_area_by_eng = sorted(area_dict.items(), key=lambda x:x[1])
area_in_korean = [area[0] for area in sorted_area_by_eng]
area_in_english = [area[1] for area in sorted_area_by_eng]
df_sorted_area_by_eng['지역명(영문)'] = area_in_english
df_sorted_area_by_eng['지역명(한글)'] = area_in_korean
df_sorted_area_by_eng.head()

In [ ]:
df_sorted_area_by_eng.to_csv('data_sorted_area_by_eng.csv')

# Tidy Data 작업
대표색 이외의 나머지 색도 16진수로 표현해두었습니다.

이상의 대표색 추출 코드를 변형했습니다.

In [ ]:
def dec_to_hex(color):
    if color < 16:
        return '0' + str(hex(int(color)).split('x')[1])
    else:
        return str(hex(int(color)).split('x')[1])

In [ ]:
def read_real_color(filename, color_rank):
    image = cv2.imread(filename, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    image_list = [str(list(image[i][k])) for i in range(len(image)) for k in range(len(image[0]))]
    image_unique = {}
    for d in image_list:
        if d not in image_unique:
            image_unique[d] = 1
        else:
            image_unique[d] += 1

    total_color_lists = sorted(image_unique.keys(), key=lambda x:image_unique[x], reverse=True)
    color_list = total_color_lists[color_rank]

    color_R = int(color_list.split('[')[1].split(']')[0].split(', ')[0])
    color_G = int(color_list.split('[')[1].split(']')[0].split(', ')[1])
    color_B = int(color_list.split('[')[1].split(']')[0].split(', ')[2])

    color_R = dec_to_hex(color_R)
    color_G = dec_to_hex(color_G)
    color_B = dec_to_hex(color_B)

    return str(color_R + color_G + color_B)

In [ ]:
for i in range(5):
    color_list = []
    for n in df_cafe.index:
        png = './cafe_color_result/' + df_cafe['파일명'][n]
        color_list.append(read_real_color(png, i))
    col_name = str(i+1) + '번 색'
    df_cafe[col_name] = color_list

In [ ]:
df_cafe.head()